# Section 1-3 - Parameter Tuning

In previous sections, we took the approach of using Scikit-learn as a black box. We now review how to tune the parameters of the model to make more accurate predictions.

## Pandas - Extracting data

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv')

df_train = df.iloc[:712, :]
df_test = df.iloc[712:, :]

## Pandas - Cleaning data

In [2]:
df_train = df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)

df_train['Embarked'] = df_train['Embarked'].fillna('S')

df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_train = pd.concat([df_train, pd.get_dummies(df_train['Embarked'], prefix='Embarked')], axis=1)

df_train = df_train.drop(['Embarked'], axis=1)

X_train = df_train.iloc[:, 2:].values
y_train = df_train['Survived']

## Scikit-learn - Training the model

The documentation for the Random Forest Classifier details the different input parameters of the model. These input parameters include the number of trees, and the number of branches each tree has. It is unclear, off-the-bat, which values would be optimal. 

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

GridSearchCV allows us to test the desired range of input parameters, and review the performance of each set of values on a cross-validation basis. Here we review the number of features considered at each step a branch is made (max_features: 50% or 100% of features) and the maximum number of branches (max_depth: 5 levels or no limitations). 

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

parameter_grid = {
    'max_features': [0.5, 1.],
    'max_depth': [5., None]
}

grid_search = GridSearchCV(RandomForestClassifier(n_estimators = 100), parameter_grid, cv=5, verbose=3)

/Users/nathalie/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nathalie/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.804196 -   0.3s
[CV] max_depth=5.0, max_features=0.5 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] ........ max_depth=5.0, max_features=0.5, score=0.797203 -   0.2s
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.888112 -   0.2s
[CV] max_depth=5.0, max_features=0.5 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s


[CV] ........ max_depth=5.0, max_features=0.5, score=0.830986 -   0.2s
[CV] max_depth=5.0, max_features=0.5 .................................
[CV] ........ max_depth=5.0, max_features=0.5, score=0.780142 -   0.2s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.797203 -   0.2s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.804196 -   0.2s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.874126 -   0.2s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.830986 -   0.2s
[CV] max_depth=5.0, max_features=1.0 .................................
[CV] ........ max_depth=5.0, max_features=1.0, score=0.794326 -   0.2s
[CV] max_depth=None, max_features=0.5 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    4.4s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_features': [0.5, 1.0], 'max_depth': [5.0, None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=3)

We now review the results.

In [5]:
grid_search.grid_scores_

[mean: 0.82022, std: 0.03773, params: {'max_depth': 5.0, 'max_features': 0.5},
 mean: 0.82022, std: 0.02993, params: {'max_depth': 5.0, 'max_features': 1.0},
 mean: 0.80197, std: 0.02963, params: {'max_depth': None, 'max_features': 0.5},
 mean: 0.79635, std: 0.03108, params: {'max_depth': None, 'max_features': 1.0}]

We now review the best-performing tuning parameters.

In [6]:
grid_search.best_params_

{'max_depth': 5.0, 'max_features': 0.5}

We then set these tuning parameters to our model.

In [7]:
model = RandomForestClassifier(n_estimators = 100, max_features=1., max_depth=5.0, random_state=0)
model = model.fit(X_train, y_train)

**Exercise**

- Write the code so that grid_search refits model with the best tuning parameters to the entire dataset after these parameters are found, and hence allow us to skip the two lines of code above.

In [8]:
model = RandomForestClassifier(n_estimators = 100,max_features=grid_search.best_params_['max_features'],max_depth=grid_search.best_params_['max_depth'], random_state=0)
model=model.fit(X_train,y_train)

## Scikit-learn - Making predictions

In [9]:
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Embarked'] = df_test['Embarked'].fillna('S')

df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)
df_test = pd.concat([df_test, pd.get_dummies(df_test['Embarked'], prefix='Embarked')],axis=1)

df_test = df_test.drop(['Embarked'], axis=1)

X_test = df_test.iloc[:, 2:]
y_test = df_test['Survived']

y_prediction = model.predict(X_test)

## Evaluation

In [10]:
np.sum(y_prediction == y_test) / float(len(y_test))

0.86592178770949724